## Here I checked redisai python client a bit.

In [1]:
from redisai import Client

### Be sure that redisai is working. Run the container as it is described at README.md

In [2]:
# be sure that ports of the container and the client are matching
con = Client(host='0.0.0.0', port=6379)

In [3]:
## In case you need to save a model in ONNX format:
'''
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# prepare the train and test data
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y)

# train a model
model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)

# convert the model to ONNX
initial_types = [('input', FloatTensorType([None, 4]))]

onnx_model = convert_sklearn(model,
                             initial_types=initial_types)

# save the model
model_path = "iris_2.onnx"
with open(model_path, "wb") as f:
    f.write(onnx_model.SerializeToString())
# load the model
model = open(model_path, 'rb').read()
'''

'\nfrom sklearn.datasets import load_iris\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.linear_model import LogisticRegression\n\nfrom skl2onnx import convert_sklearn\nfrom skl2onnx.common.data_types import FloatTensorType\n\n# prepare the train and test data\niris = load_iris()\nX, y = iris.data, iris.target\nX_train, X_test, y_train, y_test = train_test_split(X, y)\n\n# train a model\nmodel = LogisticRegression(max_iter=5000)\nmodel.fit(X_train, y_train)\n\n# convert the model to ONNX\ninitial_types = [(\'input\', FloatTensorType([None, 4]))]\n\nonnx_model = convert_sklearn(model,\n                             initial_types=initial_types)\n\n# save the model\nmodel_path = "iris_2.onnx"\nwith open(model_path, "wb") as f:\n    f.write(onnx_model.SerializeToString())\n# load the model\nmodel = open(model_path, \'rb\').read()\n'

In [4]:
iris_model = open('iris.onnx', 'rb').read()

In [5]:
# redis-cli -x AI.MODELSET model_key ONNX CPU BLOB < iris.onnx
con.modelset(key='iris_model',
             backend='ONNX',
             device='CPU',
             data=iris_model,
             outputs=['classes', 'confidence'])

'OK'

In [6]:
# scan loaded to RedisAI models
con.modelscan()

/home/az/miniconda3/envs/redisai_py/lib/python3.7/site-packages/redisai/client.py:336: UserWarning: Experimental: Model List API is experimental and might change in the future without any notice
  "in the future without any notice", UserWarning)


[['iris_model', '']]

In [7]:
# redis-cli AI.TENSORSET iris:in FLOAT 2 4 VALUES 5.0 3.4 1.6 0.4 6.0 2.2 5.0 1.5
con.tensorset(key='iris', tensor=[5.0, 3.4, 1.6, 0.4, 6.0, 2.2, 5.0, 1.5], shape=[2,4], dtype='float')

'OK'

In [8]:
# redis-cli AI.TENSORGET iris:in VALUES
con.tensorget(key='iris')

array([[5. , 3.4, 1.6, 0.4],
       [6. , 2.2, 5. , 1.5]], dtype=float32)

In [9]:
# redis-cli AI.TENSORGET iris:in META
con.tensorget(key='iris', meta_only=True)

{'dtype': 'FLOAT', 'shape': [2, 4]}

In [10]:
# redis-cli AI.TENSORGET iris:in META VALUES
con.tensorget(key='iris', as_numpy=False)

{'dtype': 'FLOAT',
 'shape': [2, 4],
 'values': [5.0,
  3.4000000953674316,
  1.600000023841858,
  0.4000000059604645,
  6.0,
  2.200000047683716,
  5.0,
  1.5]}

In [11]:
# redis-cli AI.MODELRUN iris INPUTS iris:in OUTPUTS iris:inferences iris:scores
con.modelrun(key='iris_model', inputs='iris', outputs=['class', 'confidence'])

'OK'

In [12]:
# let's get predictions.

In [13]:
con.tensorget(key='class')

array([0, 1])

In [14]:
con.tensorget(key='class', as_numpy=True, meta_only=True)

{'dtype': 'INT64', 'shape': [2]}

In [18]:
con.tensorset(key='to_predict', tensor=[5.0, 3.4, 1.6, 0.4], shape=[1,4], dtype='float')
con.modelrun(key='iris_model', inputs='to_predict', outputs=['class', 'confidence'])
con.tensorget(key='class')[0]

0

### Some problems occured with the confidence score tensor obtaining, maybe it was caused by incorrect convertion to ONNX format.

## Now I'll build a Flask app.